In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dmt.a_leadtime_sto_ambient_ft_internal_rpt
TBLPROPERTIES (
'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
 )  
AS
WITH
raw_do_data AS(
  SELECT
    a.* EXCEPT(sto_item, deliver_qty_sale, sales_unit),
    CAST (sto_item AS INT) sto_item,
    deliver_qty_sale * 
CASE 
  WHEN COALESCE(b.denomintr, 1) != 0 
  THEN COALESCE(b.numerator, 1) / COALESCE(b.denomintr, 1) 
  ELSE 1 
END AS do_qty_base
  FROM (
    SELECT a.* FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_delivery_order a
    WHERE
      calday >= '2024-01-01'
      AND IFNULL(deliver_qty_sale,0) <> 0
    QUALIFY ROW_NUMBER () OVER (PARTITION BY sto_number, sto_item, do_number, do_item ORDER BY calday DESC, creation_date DESC, crea_time DESC) = 1) a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_unit b ON a.product_id = b.product_id AND a.sales_unit = b.unit
  WHERE calday >= (date_sub(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS, (dayofweek(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) + 6) % 7)  - INTERVAL 26 WEEK)
),
do_data AS (
  SELECT
    a.sto_number,
    a.sto_item,
    a.product_id,
    a.store_id,
    a.creation_date do_creation_date,
    a.act_gi_dte,
    a.do_number,
    a.do_item,
    a.sts_pick,
    a.dc_id,
    IFNULL(a11.po_gr_date,a21.po_gr_date) po_gr_date,
    CASE
      WHEN a1.sto_number IS NOT NULL THEN 'FT_SAP'
      WHEN a2.sto_number IS NOT NULL THEN 'FT_SFT'
      ELSE 'BS'
    END sto_type,
    SUM(a.do_qty_base) do_qty,
    SUM(a.deliver_qty_base) gi_qty
  FROM
    raw_do_data a
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_allocation_table WHERE wms = 'SAP') a1 ON
      a.sto_number = a1.sto_number
      AND CAST(a.sto_item AS INT) = CAST(a1.sto_item AS INT)
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_allocation_table WHERE wms = 'SFT') a2 ON
      a.sto_number = a2.sto_number
    LEFT JOIN (SELECT DISTINCT po_doc, po_item, po_gr_date FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item
              WHERE calday >= (date_sub(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS, (dayofweek(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) + 6) % 7)  - INTERVAL 26 WEEK)) a11 ON
      a1.po_doc = a11.po_doc AND a1.po_item = a11.po_item
    LEFT JOIN (SELECT DISTINCT po_doc, po_gr_date FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_po_item
              WHERE calday >= (date_sub(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS, (dayofweek(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) + 6) % 7)  - INTERVAL 26 WEEK)) a21 ON
      a2.po_doc = a21.po_doc
  WHERE a.calday BETWEEN (date_sub(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS, (dayofweek(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) + 6) % 7)  - INTERVAL 26 WEEK) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
  GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
),
ft AS (
  SELECT
     a.do_number,
     a.do_item,
     a.sto_number,
     a.sto_item,
     a.store_id,
     a.product_id,
     a.po_gr_date AS start_date,
     a.act_gi_dte AS end_date,
     a.dc_id,
     a.sto_type,
     a.sts_pick,
     IFNULL(SUM(a.do_qty),0) do_qty,
     IFNULL(SUM(a.gi_qty),0) gi_qty
  FROM do_data a
  WHERE sto_type IN('FT_SAP', 'FT_SFT')
  GROUP BY 1,2,3,4,5,6,7,8,9,10,11
),
temp_wk AS (
  SELECT DISTINCT
    do_number,
    do_item,
    sto_type,
    sto_number,
    sto_item,
    product_id,
    end_date,
    start_date,
    store_id,
    a.dc_id,
    do_qty,
    gi_qty,
    sts_pick,
    COUNT(IF(f.workingday = 0,1,NULL)) subtract_wkd,
    COUNT(IF(g.dc_id IS NOT NULL AND g.stock_count_date = f.date,1,NULL)) subtract_stockcnt
  FROM
    ft a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_working_day_cnt f ON a.end_date >= f.date AND a.start_date <=f.date
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.d_portal_stock_count_date_bydc g ON a.end_date >= g.stock_count_date AND a.start_date <=g.stock_count_date AND a.dc_id = CAST(g.dc_id AS STRING)
  GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13
),
ontime_check AS (
  SELECT
    a.*,
    DATEDIFF(end_date, start_date) - (IFNULL(subtract_wkd,0) + IFNULL(subtract_stockcnt,0)) leadtime_actual,
    CASE WHEN city IN ('HÀ NỘI','HỒ CHÍ MINH') AND manufacturer_id = '1000' THEN 4 ELSE 7 END leadtime_qual,
    DATEDIFF(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS), start_date) - (IFNULL(subtract_wkd,0) + IFNULL(subtract_stockcnt,0)) time_tilltoday
  FROM
    temp_wk a
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store USING (store_id)
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product USING (product_id)
    INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time d ON a.start_date = d.calday
  WHERE
    business_unit IN ('1500','2000')
    AND mch2_id IN ('102','201','202','203')
),
summary AS (
  SELECT
    a.* ,
    CASE 
    WHEN leadtime_actual IS NULL THEN 
        CASE 
            WHEN IF(GREATEST(0, leadtime_actual) = 0, NULL, GREATEST(0, leadtime_actual)) <= leadtime_qual THEN 'ONTIME'
            WHEN start_date > end_date THEN 'NOT COUNT'
            WHEN time_tilltoday <= leadtime_qual THEN 'IN LEADTIME REQUIRED'
            ELSE 'LATE'
        END
    WHEN GREATEST(0, leadtime_actual) <= leadtime_qual THEN 'ONTIME'
    WHEN start_date > end_date THEN 'NOT COUNT'
    WHEN time_tilltoday <= leadtime_qual THEN 'IN LEADTIME REQUIRED'
    ELSE 'LATE'
    END AS ontime_check
  FROM ontime_check a
)
SELECT a.*,
    IF(end_date IS NOT NULL,gi_qty,0) infull_qty,
    IF(ontime_check = 'ONTIME',gi_qty,0) ontime_qty
FROM summary a;
""")